In [41]:
import subprocess
import random
import os
import pandas as pd
import shutil

Define key variables

In [42]:
#The Mach speed of the airfoil
MACH = 0.2

#The variability of the Mach speed, + or - this number
MACH_VARIABILITY = 0.001

#The Alphas of the airfoil
ALPHAS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

#The variability of the alphas
ALPHA_VARIABILITY = 0.08

#The Reynolds number
REYNOLDS_NUM = 250000

#The variability of the Reynolds number
REYNOLDS_NUM_VARIABILITY = 2500

#The list of the airfoils to run analysis on
AIRFOILS = ["NACA 0012", "NACA 1412", "NACA 2412", "NACA 3412", "NACA 4412", "NACA 5412", "NACA 6412", "NACA 7412", "NACA 8412", "NACA 9412"]

#Toggle showing the graph UI when running notebook
HIDE_GRAPH = False

#Number of iterations to find convergence
NUM_ITERATIONS = 400

#Number of simulations done using xfoil for each alpha
NUM_SIMS = 15

#Where to save the results
OUTPUT_FILE = "./outputs"

SEED = 12345

In [43]:
random.seed(SEED)

We must define a function to run the commands through XFoil. Because XFoil runs in a console, we use the subprocess library as a proxy to run the commands.

In [44]:
def run_xfoil_commands(command_list):
    """
    Run a batch of XFoil commands using subprocess and return the output.

    Args:
        command_list: The list of commands to run in XFoil

    Returns:
        The console output of XFoil after running commands
    """
    # Join the list of commands into a single string with each command separated by a newline
    commands = "\n".join(command_list)
    
    #Create a subprocess to run the XFoil commands
    process = subprocess.Popen(['./XFOIL6.99/xfoil'],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True
                               )
    
    #Run the commands and return the console ouput, aswell as any errors
    output, error = process.communicate(input=commands)
    return output, error

We must then define a function to actually create the list of commands for each airfoil

In [45]:
def create_commands(airfoil, alpha):
    """
    Creates a list of commands to be fed into the subprocess.
    Args:
        airfoil: The NACA airfoil to be used
    Returns:
        commands: The list of commands
        machs: The list of different machs generated with variability
        alphas: The list of different alphas generated with variability
    """

    #Create a temporary folder where XFoil can save data
    output_path = f"{OUTPUT_FILE}/temp"
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)

    
    machs = []
    alphas = []

    #Create a list of machs and alphas with random variability defined above
    machs = [round(random.uniform(MACH - MACH_VARIABILITY, MACH + MACH_VARIABILITY), 3) for _ in range(NUM_SIMS)]
    alphas = [round(random.uniform(alpha - ALPHA_VARIABILITY, alpha + ALPHA_VARIABILITY), 3) for _ in range(NUM_SIMS)]
    reynolds = [int(random.uniform(REYNOLDS_NUM - REYNOLDS_NUM_VARIABILITY, REYNOLDS_NUM + REYNOLDS_NUM_VARIABILITY)) for _ in range(NUM_SIMS)]

    commands = []

    #Turns off the graph window popping up when running the code
    if HIDE_GRAPH: commands.extend([
        "PLOP",
        "G F",
        ""
    ])
    
    #Initializes XFoil so it is ready to start the analysis
    commands.extend([
        airfoil,
        "OPER",
        f"ITER {NUM_ITERATIONS}",
        f"Re {REYNOLDS_NUM}"
        "VISC"
    ])

    #For each simulation, add more commands to be run in sequence, each with a randomly variated  mach and alpha number
    for i in range(NUM_SIMS):
        new_commands = [
            f"MACH {machs[i]}",
            f"RE {reynolds[i]}",
            "PACC",
            f"{output_path}/{i}.txt",
            "",
            f"ALFA {alphas[i]}",
            "PACC",
            "PDEL",
            "0"
        ]
        commands.extend(new_commands)
    
    return commands, machs, alphas

Next we have to actually read the data that XFoil saves in a text document

In [46]:
def read_output(file_path):
    """
    Parses through a XFoil save file and extracts cl, cd, cdp, cm, top_xtr, and bot_xtr
    Args:
        file_path: The path to the text file
    Returns:
        alpha: The alpha saved in the text document. This is cross-checked with the alpha defined in the list above to make sure the data lines up.
        cl: Coefficient of lift
        cd: Coefficient of drag
        cdp: Coefficient of pressure drag
        cm: Coefficient of momentum
        top_xtr: Top transition point
        bot_xtr: Bottom transition point
    """

    with open(file_path, 'r') as file:
        #Iterates through every line in the text file
        for i, line in enumerate(file, start=1):
            #Parses through the 13th line, as that is where the important values are stored
            if i == 13:
                #Splits the line by the spaces between the values and saves them accordingly
                values = line.split()
                alpha = float(values[0])
                cl = float(values[1])
                cd = float(values[2])
                cdp = float(values[3])
                cm = float(values[4])
                top_xtr = float(values[5])
                bot_xtr = float(values[6])
                return alpha, cl, cd, cdp, cm, top_xtr, bot_xtr
    return None  # Return None if the file doesn't have 13 lines

We also need to make a function that goes through every save file in the temp save folder and makes it into a pandas dataframe

In [47]:
def process_folder(machs, alphas, folder_path):
    """
    Parses through every save text file in a folder, extracts the output values, and saves them to a pandas dataframe
    Args:
        machs: The list of machs with variability defined earlier in create_commands()
        alphas: The list of alphas with variability defined earlier in create_commands()
        folder_path: The path to the folder that contains the save files
    Returns:
        A pandas dataframe containing all of the saved data
    """
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            result = read_output(file_path)
            if result:
                alpha, cl, cd, cdp, cm, top_xtr, bot_xtr = result
                index = int(os.path.splitext(filename)[0])
                mach = machs[index]
                if alpha != alphas[index]: print("Alpha Mismatch!")
                data.append([alpha, mach, cl, cd, cdp, cm, top_xtr, bot_xtr])
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Alpha', 'Mach', 'CL', 'CD', 'CDp', 'CM', 'Top_Xtr', 'Bot_Xtr'])
    return df

Finally, we have to actually run the commands

In [48]:
#Iterates through every airfoil in the list and runs an analysis on it, saving the dataframe to a csv file.
for airfoil in AIRFOILS:
    save_path = f"{OUTPUT_FILE}/{airfoil}"
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path)
    for alpha in ALPHAS:
        folder_path = f"{OUTPUT_FILE}/temp"
        commands, machs, alphas = create_commands(airfoil, alpha)
        output, error = run_xfoil_commands(commands)
        print(error)
        df = process_folder(machs, alphas, folder_path)
        shutil.rmtree(folder_path) #Removes the temparary folder when done with the airfoil
        df.to_csv(f"{save_path}/{alpha}.csv", index=True)

At line 135 of file ../src/userio.f (unit = 5, file = 'stdin')
Fortran runtime error: End of file

